In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import string
from nltk.corpus import stopwords


In [ ]:
df = pd.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")


In [ ]:
df.head(5)

In [ ]:
df.describe()

In [ ]:
df['sentiment_num'] = df.sentiment.map({'negative':0, 'positive':1})

In [ ]:
df.head()

In [ ]:
df.columns = ['comment', 'label', 'label_num']

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.groupby('label').describe()

In [ ]:
df['length'] = df['comment'].apply(len)

In [ ]:
df.head()

In [ ]:
## see what we're up against...

df[df['length']>50]['comment'].iloc[0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()

In [ ]:
def message_text_process(mess):
    STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])

In [ ]:
bag_of_words = CountVectorizer(analyzer=message_text_process).fit(df['comment'])

In [ ]:
print(len(bag_of_words.vocabulary_))

In [ ]:
print(bag_of_words.vocabulary_)

In [ ]:
message_bagofwords = bag_of_words.transform(df['comment'])

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(message_bagofwords)

In [ ]:
message_tfidf = tfidf_transformer.transform(message_bagofwords)
print(message_tfidf.shape)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
sentiment_detect_model = MultinomialNB().fit(message_tfidf,df['label'])

In [ ]:
comment = df['comment'][0]
bag_of_words_for_comment = bag_of_words.transform([comment])
tfidf = tfidf_transformer.transform(bag_of_words_for_comment)


In [ ]:
row = 23472
print('predicted', sentiment_detect_model.predict(tfidf)[0])
print('actual', df.label[row])
df[row:row+1]